In [ ]:
!pip install langchain-community

In [ ]:
!pip install pypdf
from langchain.document_loaders import PyPDFLoader

raw_text=PyPDFLoader("/content/DATA/paracetamol.pdf")
raw_text

In [ ]:
raw_file=raw_text.load()
raw_file

[Document(metadata={'source': '/content/DATA/paracetamol.pdf', 'page': 0}, page_content=' 1\nPackage leaflet: Information for the user \n \nParacetamol 500 mg tablets \nparacetamol  \n \n \nRead all of this leaflet carefully before you start taking this  medicine because it contains \nimportant information for you. \nAlways take this medicine exactly as described in this leaflet or as your doctor or pharmacist told you. \n- Keep this leaflet. You may need to read it again. \n- Ask your pharmacist if you need more information or advice. \n- If you get any side effects, talk to your doctor, or pharmacist . This includes any possible side \neffects not listed in this leaflet. See section 4. \nYou must talk to a doctor if you do not feel better or if you feel worse after 3 days of fever and 5 \ndays of pain or if other symptoms appear.   \n \nWhat is in this leaflet  \n1. What Paracetamol is and what it is used for \n2. What you need to know before you take Paracetamol \n3. How to take Par

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
documents=text_splitter.split_documents(raw_file)
documents

[Document(metadata={'source': '/content/DATA/paracetamol.pdf', 'page': 0}, page_content='1\nPackage leaflet: Information for the user \n \nParacetamol 500 mg tablets \nparacetamol  \n \n \nRead all of this leaflet carefully before you start taking this  medicine because it contains \nimportant information for you. \nAlways take this medicine exactly as described in this leaflet or as your doctor or pharmacist told you. \n- Keep this leaflet. You may need to read it again. \n- Ask your pharmacist if you need more information or advice.'),
 Document(metadata={'source': '/content/DATA/paracetamol.pdf', 'page': 0}, page_content='- Ask your pharmacist if you need more information or advice. \n- If you get any side effects, talk to your doctor, or pharmacist . This includes any possible side \neffects not listed in this leaflet. See section 4. \nYou must talk to a doctor if you do not feel better or if you feel worse after 3 days of fever and 5 \ndays of pain or if other symptoms appear.   \

In [ ]:
documents=[doc.page_content for doc in documents]
documents

['1\nPackage leaflet: Information for the user \n \nParacetamol 500 mg tablets \nparacetamol  \n \n \nRead all of this leaflet carefully before you start taking this  medicine because it contains \nimportant information for you. \nAlways take this medicine exactly as described in this leaflet or as your doctor or pharmacist told you. \n- Keep this leaflet. You may need to read it again. \n- Ask your pharmacist if you need more information or advice.',
 '- Ask your pharmacist if you need more information or advice. \n- If you get any side effects, talk to your doctor, or pharmacist . This includes any possible side \neffects not listed in this leaflet. See section 4. \nYou must talk to a doctor if you do not feel better or if you feel worse after 3 days of fever and 5 \ndays of pain or if other symptoms appear.   \n \nWhat is in this leaflet  \n1. What Paracetamol is and what it is used for \n2. What you need to know before you take Paracetamol',
 '2. What you need to know before you ta

In [ ]:
!pip install google.generativeai>=0.8.3 chromadb

In [ ]:
import google.generativeai as genai
from IPython.display import Markdown


In [ ]:
genai.configure(api_key="AIzaSyCRxDyK59tPFcWB0qQtd2h")

In [ ]:
for m in genai.list_models():
     if "embedContent" in m.supported_generation_methods:
         print(m.name)

models/embedding-001
models/text-embedding-004


In [ ]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

        response = genai.embed_content(
            model="models/text-embedding-004",
            content=documents,
            task_type=embedding_task,
            request_options=retry_policy,
        )
        return response["embedding"]

In [ ]:
import chromadb
DB_NAME = "googlecardb"
embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)

db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

In [ ]:
db.count()

214

In [ ]:
db.peek(1)

{'ids': ['0'],
 'embeddings': array([[ 1.87941343e-02,  2.12922432e-02, -3.51193212e-02,
         -7.21177738e-03,  1.64561477e-02, -2.69960258e-02,
         -2.14216970e-02,  4.17354032e-02, -4.84089740e-03,
          2.91666240e-02,  4.56957659e-03, -1.21571461e-03,
          5.37956655e-02,  6.32065535e-03,  6.03143498e-02,
         -6.87590688e-02, -6.10287674e-03,  3.96302231e-02,
         -9.05265361e-02, -2.48595085e-02,  3.53463627e-02,
          1.97532512e-02,  3.66369151e-02,  4.74498719e-02,
         -5.22560952e-03,  6.88785166e-02,  6.09978326e-02,
          2.36070212e-02, -5.70264943e-02, -3.34884524e-02,
          2.14217976e-02,  7.16584211e-04,  2.88722515e-02,
         -1.63756832e-02,  2.17096768e-02,  2.20670737e-02,
         -2.65586190e-02,  5.16547337e-02,  4.25024480e-02,
         -4.04727757e-02, -4.72782142e-02,  3.67601067e-02,
          7.65943378e-02, -1.74722392e-02, -3.19013111e-02,
          1.11471536e-02, -3.26592140e-02,  9.23342071e-03,
         -4

In [ ]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "What are the contents of Paracetamol?"

result = db.query(query_texts=[query], n_results=1)
result

{'ids': [['197']],
 'embeddings': None,
 'documents': [['6. Contents of the pack and other information \n \nWhat Paracetamol contains']],
 'uris': None,
 'data': None,
 'metadatas': [[None]],
 'distances': [[0.429695188999176]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [ ]:
[[passage]] = result["documents"]
Markdown(passage)

6. Contents of the pack and other information 
 
What Paracetamol contains

In [ ]:
passage_oneline = passage.replace("\n", " ")
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""You are a helpful and informative bot that answers questions using text from the reference passage included below.
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.

QUESTION: {query_oneline}
PASSAGE: {passage_oneline}
"""
print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below.
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.

QUESTION: What are the contents of Paracetamol?
PASSAGE: 6. Contents of the pack and other information    What Paracetamol contains



In [ ]:
model = genai.GenerativeModel("gemini-1.5-flash-latest")
answer = model.generate_content(prompt)
Markdown(answer.text)

ERROR:tornado.access:503 POST /v1beta/models/gemini-1.5-flash-latest:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 7940.40ms


This question cannot be answered from the given text.  The provided text only indicates that there is a section titled "What Paracetamol contains," but it does not actually list the contents of Paracetamol.
